In [1]:
import time
import pandas as pd
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException

In [3]:
#Find driver at: https://sites.google.com/chromium.org/driver/downloads
driver = webdriver.Chrome()
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

In [ ]:
driver = webdriver.Chrome(options=options)

In [ ]:
energyEfficiencyBaseURL = 'https://energiforskning.dk/projekter?search_api_fulltext=&f%5B0%5D=t%3A95&page={}'
smartGridsBaseURL = 'https://energiforskning.dk/projekter?search_api_fulltext=&f%5B0%5D=t%3A98&page={}'
otherBaseURL = 'https://energiforskning.dk/projekter?search_api_fulltext=&f%5B0%5D=t%3A99&page={}'

In [ ]:
def energiforskningWebscraper(baseUrl):
    
    projectTitle = []
    projectURL = []
    
    
    page = 0
    
    def elementExists(driver, locator):
        try:
            driver.find_element_by_css_selector(locator)
            return True
        except NoSuchElementException:
            return False
    
    while True:
        
        driver.get(baseUrl.format(page))
        
        if elementExists(driver, 'body > div > main > div > div > article > div > div.alpha > div.field.field-viewsreference > div > div > div.view-content'):
            
            _projects = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div > div.alpha > div.field.field-viewsreference > div > div > div.view-content')
            _projects = _projects.find_elements(By.CLASS_NAME, 'views-row')
            
            for _project in _projects:
                try:

                    _title = _project.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div > div.alpha > div.field.field-viewsreference > div > div > div.view-content > div > div > div > div.field.node-title > h2 > a').text
                    _url = _project.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div > div.alpha > div.field.field-viewsreference > div > div > div.view-content > div > div > div > div.field.node-title > h2 > a').get_attribute('href')

                    projectTitle.append(_title)
                    projectURL.append(_url)

                except Exception as e:
                    
                    print(e)
                    print(page)
                    print(_project.text)
                    
        else:
            break

        page = page + 1
    
    df = pd.DataFrame({'Project Title': projectTitle, 'URL' : projectURL})
    
    return df

In [ ]:
smartGridProjects = energiforskningWebscraper(smartGridsBaseURL)
energyEfficiencyProjects = energiforskningWebscraper(energyEfficiencyBaseURL)
otherProjects = energiforskningWebscraper(otherBaseURL)

In [ ]:
smartGridProjects['Category'] = 'Smart Grid & Systems'
energyEfficiencyProjects['Category'] = 'Energy Efficiency'
otherProjects['Category'] = 'Other'

In [ ]:
energiforskningProjects = pd.concat([smartGridProjects,energyEfficiencyProjects,otherProjects], ignore_index=True)

In [ ]:
def elementExists(driver, locator):
    try:
        driver.find_element_by_css_selector(locator)
        return True
    except NoSuchElementException:
        return False

In [ ]:
projectDescriptions = []
projectKeyFigures = []
projectPrograms = []
projectMainResponsible = []
projectParticipants = []

for url in tqdm(energiforskningProjects['URL'].to_list()):
    
    driver.get(url)

    _description = 'n/a'
    _keyFigures = 'n/a'
    _program = 'n/a'
    _mainResponsible = 'n/a'
    _participants = 'n/a'

    if elementExists(driver, 'body > div > main > div > div > article > div.inner > div.alpha > div.field.field-content'):
        _description = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div.inner > div.alpha > div.field.field-content').text

    if elementExists(driver, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.numbers_wrapper > div'):
        _keyFigures = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.numbers_wrapper > div').text

    if elementExists(driver, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.category_wrapper > div.field.field-program-ref.field--label-inline'):
        _program = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.category_wrapper > div.field.field-program-ref.field--label-inline').text

    if elementExists(driver, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.participants_wrapper > div.field.octo-main-resposible'):
        _mainResponsible = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.participants_wrapper > div.field.octo-main-resposible').text

    if elementExists(driver, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.participants_wrapper > div.field.field-partners > table > tbody'):
        _participants = driver.find_element(By.CSS_SELECTOR, 'body > div > main > div > div > article > div.inner > div.alpha > div.details > div.participants_wrapper > div.field.field-partners > table > tbody').text

    projectDescriptions.append(_description)
    projectKeyFigures.append(_keyFigures)
    projectPrograms.append(_program)
    projectMainResponsible.append(_mainResponsible)
    projectParticipants.append(_participants)